In [13]:
import json
import pandas as pd
import re
import os
from collections import Counter
from pathlib import Path

In [14]:
# Baca data dulu ke dalam df
df = pd.read_csv("D:\SEMESTER 6\PROJECT CBR\putusan_ma__2025-06-23.csv") 

<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
C:\Users\ASUS\AppData\Local\Temp\ipykernel_26888\1433169407.py:2: SyntaxWarning: invalid escape sequence '\S'
  df = pd.read_csv("D:\SEMESTER 6\PROJECT CBR\putusan_ma__2025-06-23.csv")


In [15]:
print("Jumlah nilai kosong per kolom:")
print(df.isna().sum())


Jumlah nilai kosong per kolom:
judul                        0
nomor                        0
tingkat_proses               0
klasifikasi                  0
kata_kunci                   0
tahun                        0
tanggal_register             0
lembaga_peradilan            0
jenis_lembaga_peradilan      0
hakim_ketua                  0
hakim_anggota                0
panitera                     0
amar                         0
amar_lainnya                 7
catatan_amar                 0
tanggal_musyawarah           0
tanggal_dibacakan            0
kaidah                       0
status                     112
abstrak                    112
link                         0
link_pdf                     3
file_name_pdf                3
text_pdf                     3
dtype: int64


In [22]:
# Isi missing value dengan default / placeholder
df['amar_lainnya'] = df['amar_lainnya'].fillna("-").astype(str)
df['status'] = df['status'].fillna("tidak tersedia").astype(str)
df['abstrak'] = df['abstrak'].fillna("tidak tersedia").astype(str)
df['link_pdf'] = df['link_pdf'].fillna("tidak ada").astype(str)
df['file_name_pdf'] = df['file_name_pdf'].fillna("-").astype(str)

In [23]:
# Util untuk bersihkan boiler-plate
def clean_header(text: str) -> str:
    text = str(text)                               # ← konversi pasti string
    text = re.sub(r"Mahkamah Agung.*?\n", " ", text, flags=re.I|re.S)
    text = re.sub(r"Direktori Putusan.*?\n", " ", text, flags=re.I|re.S)
    text = re.sub(r"Halaman \d+\s*", " ", text, flags=re.I)
    return text

In [24]:
def argumen_hukum(teks):
    kalimat = [k for k in str(teks).split('. ') if 'pasal' in k.lower()]
    return '. '.join(kalimat[:2]).strip()

def jumlah_kata(teks):
    return len(str(teks).split())

def bag_of_words(teks, top_n=5):
    words = re.findall(r'\b\w+\b', str(teks).lower())
    stop = {'yang','dan','di','ke','dengan','untuk','dari','pada','adalah'}
    words = [w for w in words if w not in stop]
    return ', '.join([w for w,_ in Counter(words).most_common(top_n)])

def qa_pair(fakta, amar):
    if not fakta or not amar:
        return "-"
    return f"Q: Apa yang terjadi dalam kasus ini?\nA: {fakta}\n\n" \
            f"Q: Apa hasil putusannya?\nA: {amar}"

In [25]:
# Tambah kolom baru
df["ringkasan_fakta"] = df["text_pdf"].apply(ringkasan_fakta)
df["argumen_hukum"]  = df["text_pdf"].apply(argumen_hukum)
df["text_length"]    = df["text_pdf"].apply(jumlah_kata)
df["bag_of_words"]   = df["text_pdf"].apply(bag_of_words)
df["qa_pair"]        = df.apply(lambda x: qa_pair(x["ringkasan_fakta"], x["amar"]), axis=1)
df["case_id"]        = df.index + 1

In [26]:
cols = ["case_id","nomor","tanggal_register","klasifikasi","ringkasan_fakta",
        "argumen_hukum","amar","hakim_ketua","text_pdf","text_length",
        "bag_of_words","qa_pair"]

Path(r"D:/SEMESTER 6/PROJECT CBR/data/processed").mkdir(parents=True, exist_ok=True)
df[cols].to_csv(r"D:/SEMESTER 6/PROJECT CBR/data/processed/cases.csv", index=False, encoding="utf-8")
df[cols].to_json(r"D:/SEMESTER 6/PROJECT CBR/data/processed/cases.json",
                    orient="records", force_ascii=False, indent=2)

print("✅ Kolom ringkasan_fakta berhasil dibuat tanpa NaN, file CSV & JSON disimpan.")

✅ Kolom ringkasan_fakta berhasil dibuat tanpa NaN, file CSV & JSON disimpan.
